In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb

In [3]:
df = pd.read_csv(r'./data/preprocess.csv')

df.head()

,IDX,mean_ndvi_2022-06-02,mean_ndvi_2022-06-17,mean_ndvi_2022-06-07,mean_ndvi_2022-06-22,mean_ndvi_2022-06-12,mean_ndre_2022-06-02,mean_ndre_2022-06-17,mean_ndre_2022-06-07,mean_ndre_2022-06-12,IDX.1,sum_ndvi_2022-06-02,sum_ndvi_2022-06-17,sum_ndvi_2022-06-07,sum_ndvi_2022-06-22,sum_ndvi_2022-06-12,sum_ndre_2022-06-02,sum_ndre_2022-06-17,sum_ndre_2022-06-07,sum_ndre_2022-06-12
0,3942,0.007476,0.212770,0.180844,0.038478,0.219896,-0.017073,0.151714,0.123746,0.153333,3942,0.530825,15.106694,12.839933,2.731913,15.612608,-1.212181,10.771678,8.785966,10.886635
1,4053,0.008709,0.202058,0.165046,-0.019584,0.198812,-0.007738,0.134886,0.106043,0.128798,4053,1.689533,39.199349,32.018860,-3.799293,38.569600,-1.501124,26.167940,20.572346,24.986805
2,4196,0.204576,0.192939,0.156745,-0.011448,0.186642,0.226347,0.128658,0.095656,0.117090,4196,19.639275,18.522111,15.047543,-1.099016,17.917662,21.729303,12.351145,9.183014,11.240659
3,4422,-0.001838,0.198717,0.181343,-0.002894,0.215428,-0.036878,0.132829,0.113879,0.136089,4422,-0.104746,11.326846,10.336535,-0.164956,12.279409,-2.102046,7.571269,6.491099,7.757051
4,4424,-0.046762,0.205482,0.185708,-0.005377,0.218237,-0.094255,0.145178,0.124556,0.148208,4424,-1.730197,7.602816,6.871180,-0.198932,8.074769,-3.487440,5.371600,4.608560,5.483689


In [74]:
df = df.drop( columns=['IDX', 'IDX.1'] )

In [75]:
target = pd.read_csv('./data/target_PROD.csv')

In [76]:
l = target[['PROD']]

str(list(l['PROD']))

'[31595, 3285, 2581, 13155, 2487, 0, 3843, 2537, 14912, 10374, 5482, 6076, 4489, 1235, 12240, 10599, 4426, 691, 5631, 2173, 1729, 12134, 3248, 4940, 4322]'

In [77]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = pd.read_csv(r'./data/preprocess.csv')
y = target['PROD']

X_train, X_test, y_train, y_test= train_test_split(
    
    X,y,
    train_size   = 0.8,
    random_state = 1234,
    shuffle      = True
)

In [86]:
import warnings

warnings.filterwarnings('ignore')

In [87]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

param_grid = {
    
    'hidden_layer_sizes': [(150,100,50), (120,80,40), (100,50,30)],
    'max_iter': [50, 100],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

grid = GridSearchCV(MLPRegressor(), param_grid, n_jobs= -1, cv=5)

grid.fit(X_train, y_train)

# que diferencia hay entre este cv y el que hago en el pipeline ¿?

GridSearchCV(cv=5, estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(150, 100, 50), (120, 80, 40),
                                                (100, 50, 30)],
                         'learning_rate': ['constant', 'adaptive'],
                         'max_iter': [50, 100], 'solver': ['sgd', 'adam']})

In [89]:
print(grid.best_params_) 

{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (120, 80, 40), 'learning_rate': 'adaptive', 'max_iter': 50, 'solver': 'sgd'}


In [88]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import KFold

In [67]:
scaler=StandardScaler()
x_train=scaler.fit_transform(X_train)
x_test=scaler.transform(X_test)

In [91]:
mlp=MLPRegressor(**grid.best_params_)

pipeline=Pipeline([('transformer',scaler),('estimator',mlp)])

pipeline.fit(X,y)

cv=KFold(n_splits=5)

# el cv lo aplicamos a todo el pipeline:

scores_crossval=cross_val_score(pipeline,x_test,y_test,cv=cv)

In [84]:
# Score and prediction:

score = pipeline.score(X_test, y_test)

predicted_y = pipeline.predict(X_test)

In [80]:
print('scores cross val',scores_crossval)

print('pipeline score',pipeline.score(x_test,y_test))

scores cross val [-13.27685809  -2.20996751          nan]
pipeline score -1.262720586100106


In [85]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predicted_y))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predicted_y))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted_y)))

Mean Absolute Error: 5165.410953704504
Mean Squared Error: 42994093.74231811
Root Mean Squared Error: 6556.98816091032
